In [1]:
import pandas as pd
import scipy.io as io
import autograd.numpy as np
from scipy.optimize import minimize

In [2]:
################### Initialization ########################

In [3]:
# load data
data = io.loadmat('./iv.mat')
ps2 = io.loadmat('./ps2.mat')

# extract data
iv = data['iv']
x1 = ps2['x1'].toarray()
x2 = ps2['x2']
IV = np.concatenate((iv[:,1:],x1[:,1:]),axis=1)
# observed market share
s_jt = ps2['s_jt']
v = ps2['v']
demogr = ps2['demogr']

# 20 random draws
ns = 20
# 94 markets
nmkt = 94
# 24 products in each market
nbrn = 24

# Initial theta_2
# parameters that start with 0 will NOT be optimized
theta2w =  np.array([0.3302, 5.4819, 0, 0.2037, 0,
                         2.4526, 15.8935, -1.2000, 0, 2.6342,
                         0.0163, -0.2506, 0, 0.0511, 0,
                         0.2441, 1.2650, 0, -0.8091, 0], ndmin=2)




In [4]:


class NEVO():
    '''
    A simple replication of NEVO's RTE cereal industry paper. Refer to https://rasmusen.org/zg604/lectures/blp/frontpage.htm for more details.
    '''
    def __init__(self, nmkt, nbrn, ns, x1, x2, Z, share, v, demogr, theta2w):
        '''
        Initialize the NEVO's problem using the fake-RTE cereal data. 
        
        -----------------------------------
        nmkt : number of market
        nbrn : number of brands. For simplicity, each brand exists in each market
        ns : number of individual draws in each market
        x1 : product features in delta_{jt}
        x2 : product features in mu_{ijt}
        Z : IV. Including exog. product features and additional instruments
        share : observed market share
        v : randomly drawn iid normal 
        demogr : individual demographics
        theta2w : initial guess of theta_2 (nonlinear parameters)
        ------------------------------------
        '''
        
        self.ns = ns        
        self.nmkt = nmkt     
        self.nbrn = nbrn     
        self.N = nmkt * nbrn
        self.IV = Z
        self.share = share
        self.v = v
        self.demogr = demogr
        self.x1 = x1
        self.x2 = x2
        # total number of theta_2 = K*J, including zeros
        self.K = x2.shape[1]
        self.s = share
        self.iter = 0
        self.theta2w = theta2w.reshape((4, 5))
        self.J = self.theta2w.shape[1]
        self.mask = self.theta2w !=0
        
        # this vector relates each observation to the market it is in
        self.cdid = np.array([[i]*nbrn for i in range(nmkt)]).reshape(self.N)


        # extract nonzero params
        self.theta2 = theta2w[np.where(theta2w != 0)]
        # The last index of each market
        self.cdindex = np.array([i for i in range((self.nbrn - 1),self.N, self.nbrn)])
        
        self.init_delta()
        self.old_delta = self.delta0.copy()
    def init_delta(self):
        '''
        Initialize delta using plain logit
        '''
        # create weight matrix
        self.invA = np.linalg.inv(IV.T@IV)
        # outside good share
        outshr = 1 - self.s.reshape(self.nmkt,-1).sum(axis=1)
        
        # explicit form for delta
        y = np.log(self.s.reshape(self.nmkt,-1)) - np.log(outshr).reshape(-1,1)
        
        y = y.reshape(-1,)
        # the linear projection
        mid = self.x1.T@self.IV @ self.invA@self.IV.T
        t = np.linalg.inv(mid@self.x1)@mid@y

        # initial delta
        self.delta0 = (self.x1@t).reshape(self.nmkt,self.nbrn)
        # initial objective function value
        self.gmmvalold = 0
        self.gmmdiff = 1


        
        
        
        
    def pred_share(self, delta, theta2mat):
        '''
        Predict the market share for each product in each market
        
        ---------------------------------------------------------
        delta : average utility for each market-product pair
        theta2mat : params to pass, constructed in matrix form
        
        Returns
        ----------------------------
        nd.array : predicted market share
            in shape (nmkt, nbrn)
        '''
        pi = theta2mat[:,1:]
        sigma = theta2mat[:,0]
        dem_cube = self.demogr.reshape(self.nmkt, -1, self.ns)
        mu0 = pi@dem_cube
        
        v_cube = self.v.reshape(self.nmkt,-1,self.ns)
        mu1 = np.diag(sigma)@v_cube
        mu = mu0 + mu1
        # as all brands exist in all markets, I can vectorize them into a higher-order dimensional matrix for faster processing
        mu = (self.x2.reshape(self.nmkt, self.nbrn,-1)) @ mu
        # numerator
        num = np.exp(delta.reshape(nmkt,nbrn,1) + mu)
        # denominator
        denom = 1 + num.sum(axis=1,keepdims=True)
        # purchase probability for each individual 
        self.ind_share = num/denom
        # individual purchase probability aggregated into market level
        shat = (num/denom).sum(axis=2)/self.ns
        return shat
    
    

    def update_delta(self,theta2mat):
        '''
        Find the unique delta given theta_2 and market share using contract mapping theorem
        
        ---------------------------------------------------------------------------------------
        theta2mat : params to pass, constructed in matrix form
        
        Returns
        --------------------------------------------------
        nd.array : unique average utility, delta
            shape (nmkt,nbrn)
        '''
        if self.gmmdiff < 1e-6:
            etol = self.etol = 1e-13
        elif self.gmmdiff < 1e-3:
            etol = self.etol = 1e-12
        else:
            etol = self.etol = 1e-9
        while True:
            shat = self.pred_share(self.old_delta,theta2mat)
            new_delta = self.old_delta + np.log(self.s.reshape(self.nmkt,self.nbrn)) - np.log(shat)
            if np.max(np.abs(new_delta - self.old_delta)) < etol:
                return new_delta
            self.old_delta = new_delta
            
            
    def gmmobj(self, params0, optimal_GMM = True):
        '''
        Objective function to be optimized. For this demo, 
        I only include the demand-side objective. It is totally fine to include 
        the supply side pricing rule given more structure on the system
        
        -----------------------------------------------------------------------
        param0 : initial parameters
        optimal_GMM : Two types of GMM objective function is allowed : 
            two-step efficient GMM and one-step GMM
        
        Returns
        --------------------------------------------------------------------------
        scalar
            value of the objective GMM function
        '''
        self.iter+=1
        theta_mat = np.zeros(shape=(4,5))
        theta_mat[self.mask] = params0
        delta = self.update_delta(theta_mat)
            
        # To reinforce code robustness
        if np.max(np.isnan(delta)) == 1:
            res = 1e8
        
        # Invert theta_1 using explicit form
        if optimal_GMM == False:
            theta1 = np.linalg.inv(self.x1.T@self.IV@self.IV.T@self.x1)@self.x1.T@self.IV@self.IV.T@(delta.reshape(-1,))
        else:
            theta1 = np.linalg.inv(self.x1.T@IV@self.invA@self.IV.T@self.x1)@self.x1.T@self.IV@self.invA@self.IV.T@(delta.reshape(-1,1))
        
        # record estimated theta_1
        self.theta1 = theta1
        # record the structural error
        self.omega = delta.reshape(-1,1) - self.x1@theta1
        
        res = self.omega.T@self.IV@self.invA@self.IV.T@self.omega
        if self.iter%10 == 0:
            print('GMM objective: ',res[0,0])
        # record the objective function value
        self.gmmvalnew = res[0,0]
        self.gmmdiff = np.abs(self.gmmvalold - self.gmmvalnew)
        # override the value function
        self.gmmvalold = self.gmmvalnew
        return res[0, 0]
      
        
        
    def optimize(self, opt_func, param_vec, args=(), method='BFGS'):
        '''
        Optimize the objective function
        
        Parameters
        --------------------------------------------
        opt_func : function to be optimized
        param_vec : parameters to be passed
        args: additional arguments
        method: algorithm to optimize
        
        Returns
        ---------------------------------
        nd.array:
            vector, the optimal parameters found
        '''
        success = False
        while not success:
            res = minimize(opt_func, param_vec, args=args, method='Nelder-Mead')
            param_vec = res.x
            if res.success:
                self.theta2 = res.x
                print('Optimization success!')
                break
            

  
    def jacob(self):
        '''
        Calculate the Jacobian matrix of the value function
            
        Returns
        ------------------------------------------------------------
        nd.array : Jacobian matrix w.r.t. the nonlinear parameters theta_2
            
                shape (nmkt*nbrn, ns)
        '''
        # individual purchase prob
        shares = self.ind_share.reshape(-1, self.ns)

        
        f1 = np.zeros((self.N, self.K * self.J))
    
        # partial s over partial sigma
        for i in range(self.K):
            xv = np.multiply(self.x2[:, i].reshape(self.nbrn * self.nmkt, 1) * np.ones((1, self.ns)),
                             self.v[self.cdid, self.ns * i:self.ns * (i + 1)])
            temp = np.cumsum(np.multiply(xv, shares), 0)
            sum1 = temp[self.cdindex, :]
            sum1[1:sum1.shape[0], :] = np.diff(sum1.T).T
            f1[:, i] = (np.mean((np.multiply(shares, xv - sum1[self.cdid, :])).T, 0).T).flatten()
      

    
        # partial s over partial pi
        for j in range(1,self.J):
            d = self.demogr[self.cdid, self.ns * (j-1):(self.ns * (j))]
            temp1 = np.zeros((self.N, self.K))
            for i in range(self.K):
                xd = np.multiply(self.x2[:, i].reshape(self.N, 1) * np.ones((1, self.ns)), d)
                temp = np.cumsum(np.multiply(xd, shares), 0)
                sum1 = temp[self.cdindex, :]
                sum1[1:sum1.shape[0], :] = np.diff(sum1.T).T
                temp1[:, i] = (np.mean((np.multiply(shares, xd - sum1[self.cdid, :])).T, 0).T).flatten()
            f1[:, (self.K * (j)):(self.K * (j + 1))] = temp1

        # store index for each nonzero entry(columnwise)
        rel = np.arange(self.K * self.J)[self.mask.T.flatten()]
        f = np.zeros((self.N, len(rel)))
        
        # implicit function derivative, partial delta over partial theta_2
        n_idx = 0
        for i in range(self.nmkt):
            temp = shares[n_idx:(self.cdindex[i] + 1), :]
            H1 = temp @ temp.T
            H = (np.diag(np.array(sum(temp.T)).flatten()) - H1) / self.ns
            # solve linear system instead of elementwise or inv multiplication
            f[n_idx:(self.cdindex[i] + 1), :] = -np.linalg.solve(H, f1[n_idx:(self.cdindex[i] + 1), :][:, rel])
            n_idx = self.cdindex[i] + 1
        return f
    
    def varcov(self):
        '''
        Calcualte the Variance-Covariance matrix, used when the optimization converges.
        
        Returns 
        ---------------------------------------------------------------------------------
        nd.array : Variance-Covariance matrix of all parameters
            shape (# of theta1 + # of theta2, # of theta1 + # of theta2)
        '''
        temp = self.jacob()
        a = np.concatenate((self.x1, temp), 1).T@self.IV
        IVres = np.multiply(self.IV, self.omega * np.ones((1,self.IV.shape[1] )))
        b = IVres.T@ IVres
        inv_aAa = np.linalg.inv(a@self.invA@a.T)
        f = inv_aAa @ a @ self.invA@ b@self.invA @ a.T @ inv_aAa
        return f
    
    def summary(self):
        '''
        Extract and summarize the coef. and corresponding standard error 
        
        Returns
        ---------------------------------------------------------------
        nd.array, nd.array : vector where each entry is an estimated coef., starting with linear parameters; vector where each entry is a SE
            shape (# of theta1 + # of theta2,)
        '''
        coef1 = self.theta1.flatten()
        theta_mat = np.zeros(shape=(4,5))
        theta_mat[self.mask] = self.theta2
        coef2 = theta_mat.T.flatten()[np.arange(self.J * self.K)[self.mask.T.flatten()]]
        coef = np.concatenate([coef1,coef2])
        vcov = self.varcov()
        se = np.sqrt(np.diag(vcov))
        
        # Put results into a nice DataFrame
        results = pd.DataFrame({
            "Coefficient": coef,
            "Std. Error": se
        })

        print("---------------------Linear Parameters------------------------------")
        print(results[:len(coef1)].to_string(index=True, float_format="{:.4f}".format))
        print("---------------------Non-linear Parameters------------------------------")
        print(results[len(coef1):].to_string(index=True, float_format="{:.4f}".format))
        print("-----------------------------------------------------------------------")
        return results

In [5]:
nevo = NEVO(nmkt, nbrn, ns,x1,x2,IV, s_jt, v, demogr, theta2w)

In [6]:
nevo.optimize(nevo.gmmobj, nevo.theta2)

GMM objective:  29.480363948112725
GMM objective:  28.476307111067662
GMM objective:  27.771413629562705
GMM objective:  27.51509737334278
GMM objective:  26.421464052442218
GMM objective:  25.007454999304883
GMM objective:  23.933660284396236
GMM objective:  24.03043297843372
GMM objective:  24.283337741227196
GMM objective:  23.27681132211089
GMM objective:  23.301611529262836
GMM objective:  22.89258023896531
GMM objective:  22.44694187619745
GMM objective:  22.1754119912452
GMM objective:  21.404574849531485
GMM objective:  20.59078750753995
GMM objective:  20.404673404013224
GMM objective:  20.029324051901746
GMM objective:  19.645689206857874
GMM objective:  19.709455673977747
GMM objective:  19.607915849303044
GMM objective:  19.117118575356344
GMM objective:  18.66698980609064
GMM objective:  18.826824425968205
GMM objective:  19.11770772252142
GMM objective:  18.325505221515243
GMM objective:  18.250109330685127
GMM objective:  17.94883241621703
GMM objective:  17.839349954585

GMM objective:  9.367735824487342
GMM objective:  9.292383455897564
GMM objective:  9.345003938914822
GMM objective:  9.29392772801311
GMM objective:  9.281754102578777
GMM objective:  9.2942308989479
GMM objective:  9.265394794462658
GMM objective:  9.272360895634263
GMM objective:  9.258319745966862
GMM objective:  9.262789507432103
GMM objective:  9.249837189446092
GMM objective:  9.250855616045731
GMM objective:  9.241505102766112
GMM objective:  9.233682492507484
GMM objective:  9.237382739765227
GMM objective:  9.233589486402172
GMM objective:  9.231440183783723
GMM objective:  9.231867538282494
GMM objective:  9.226683529338018
GMM objective:  9.23042561925265
GMM objective:  9.2267080900324
GMM objective:  9.223457135625175
GMM objective:  9.225278790144877
GMM objective:  9.544214399304309
GMM objective:  11.677308722636266
GMM objective:  9.974372839526708
GMM objective:  9.437017996512065
GMM objective:  9.2485462027342
GMM objective:  9.162667391355752
GMM objective:  9.163

GMM objective:  7.13107794215912
GMM objective:  7.131067932710367
GMM objective:  7.131072682338705
GMM objective:  7.131062189899922
GMM objective:  7.131057390601567
GMM objective:  7.1310564477489535
GMM objective:  7.131052168262285
GMM objective:  7.1310455762627205
GMM objective:  7.131041791301214
GMM objective:  7.131042130377644
GMM objective:  7.131032904810555
GMM objective:  7.131031069903349
GMM objective:  7.1310173250485045
GMM objective:  7.130998369089743
GMM objective:  7.130986641856838
GMM objective:  7.130959081341038
GMM objective:  7.130907089138557
GMM objective:  7.130904986006942
GMM objective:  7.130835782166761
GMM objective:  7.13079392637768
GMM objective:  7.1307675377072846
GMM objective:  7.1307393518745705
GMM objective:  7.130650040131759
GMM objective:  7.13059476030991
GMM objective:  7.130539223300822
GMM objective:  7.130446966829863
GMM objective:  7.130311707417248
GMM objective:  7.13020149369143
GMM objective:  7.129963331201948
GMM objective

GMM objective:  6.246289124570522
GMM objective:  6.232563491232737
GMM objective:  6.215455298791878
GMM objective:  6.16843915168373
GMM objective:  6.095277577625242
GMM objective:  6.083680621887289
GMM objective:  5.985649404821363
GMM objective:  5.932941803087354
GMM objective:  5.897958303655764
GMM objective:  5.836422451526946
GMM objective:  5.685741052550822
GMM objective:  5.671130795304934
GMM objective:  5.647656817878436
GMM objective:  5.546315723499494
GMM objective:  5.513687231511125
GMM objective:  5.429525764927039
GMM objective:  5.420807758697028
GMM objective:  5.371726978827876
GMM objective:  5.307749756998269
GMM objective:  5.2729924941520325
GMM objective:  5.203501246307011
GMM objective:  5.186380919477218
GMM objective:  5.118695327165329
GMM objective:  5.1860566751729875
GMM objective:  5.0922827259317
GMM objective:  5.064215065982447
GMM objective:  4.937704978059436
GMM objective:  4.8904542155244055
GMM objective:  4.921895603915805
GMM objective:

In [7]:
results = nevo.summary()

---------------------Linear Parameters------------------------------
    Coefficient  Std. Error
0      -62.8120     14.8358
1       -2.5010      0.8596
2        2.3296      0.7065
3       -0.2079      0.7723
4       -1.2022      0.5016
5        3.4846      0.6106
6        1.3882      0.6370
7       -0.4976      0.7618
8       -0.9045      0.4711
9        1.0150      0.6458
10      -0.6712      0.5656
11       1.1151      0.5934
12      -2.2484      0.7807
13      -0.1516      0.7503
14       1.2843      0.6317
15       1.7260      0.6343
16       3.5597      0.8768
17       0.0403      0.5713
18      -1.1368      0.4815
19       1.7638      0.6630
20       1.6040      0.4898
21       2.3495      0.6576
22      -0.8442      0.4964
23       0.7348      0.5884
24      -0.6848      0.5916
---------------------Non-linear Parameters------------------------------
    Coefficient  Std. Error
25       0.5587      0.1628
26       3.3174      1.3435
27      -0.0058      0.0135
28       0.0938   